In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D, SimpleRNN
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import math
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import random

In [3]:
df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Soft Computing/Assignment 3/Dataset 1.csv', engine = 'python')
df= df.sample(frac=1).reset_index(drop= True) 

# Data Preprocessing

In [4]:
# removing mentions, links, tags

def removing_mentions(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)     # removing @mentions
    text = re.sub(r'@[A-Za-zA-Z0-9]+', '', text)  # removing @mentions 
    text = re.sub(r'@[A-Za-z]+', '', text)        # removing @mentions
    text = re.sub(r'@[-)]+', '', text)            # removing @mentions

    return text

def removeing_links(text):
    text = re.sub(r'https?\/\/\S+', '', text)     # removing the hyper link
    text = re.sub(r'http?\/\/\S+', '', text)     # removing the hyper link
    
    return text

def removing_tags(text):
    text = re.sub(r'#', '', text )              # removing '#' sign
    text = re.sub(r'RT[\s]+', '', text)         # removing RT
    text = re.sub(r'&[a-z;]+', '', text)        # removing '&gt;'
    
    return text

In [5]:
# removing stopwords, punctuations

import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))  
  words = word_tokenize(text)
    #words = tokenizer.tokenize(text)
  filtered_words = [w for w in words if not w in stop_words]

  str = " "
  return str.join(filtered_words)


def remove_punctuations(text):
    words = word_tokenize(text)
    
    filtered_words = [w for w in words if not w in string.punctuation]
    
    str = " "
    return str.join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Before preprocessing
df[['message', 'label']].iloc[range(5)]

,message,label
0,Depression and anxietyThem shits are bitches,1
1,@worodal7ob ???? ????? ???,0
2,@pokerfacexxx i hate death metal,0
3,Happy 5 months babe!(: I love you so much and ...,0
4,is gonna hang out with my two favorite people ...,0


In [7]:
# Applying preprocessing

df['message'] = df['message'].apply(removing_mentions)
df['message'] = df['message'].apply(removeing_links)
df['message'] = df['message'].apply(removing_tags)
df['message'] = df['message'].apply(remove_stopwords)
df['message'] = df['message'].apply(remove_punctuations)


In [8]:
# After preprocessing
df[['message', 'label']].iloc[range(5)]

,message,label
0,Depression anxietyThem shits bitches,1
1,,0
2,hate death metal,0
3,Happy 5 months babe I love much make HAPPIEST ...,0
4,gon na hang two favorite people http //plurk.c...,0


In [9]:
print(df['label'].value_counts(0))


0    8000
1    2314
Name: label, dtype: int64


# Word Embedding

In [15]:
dataset = df
unique_label = np.unique(df["label"])
num_classes = len(unique_label)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset['message'].values, pd.get_dummies(dataset['label']).values, test_size=0.20)

MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# print(word_index)


train_seq = tokenizer.texts_to_sequences(X_train)
test_seq = tokenizer.texts_to_sequences(X_test)
pad_train = pad_sequences(train_seq, maxlen=MAX_SEQUENCE_LENGTH)
pad_test = pad_sequences(test_seq, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', pad_train.shape)

Found 17840 unique tokens.
Shape of data tensor: (8251, 250)


# Bidirectional RNN

In [16]:
from keras import layers
model = Sequential()
model.add(layers.Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=pad_train.shape[1]))
#model.add(layers.Bidirectional(layers.SimpleRNN(10, return_sequences=True), input_shape=(5, 10)))
model.add(layers.Bidirectional(layers.SimpleRNN(100, activation= 'relu', return_sequences = True, dropout = 0.5)))
model.add(layers.Bidirectional(layers.SimpleRNN(100, activation= 'relu', return_sequences=True, dropout = 0.5)))
model.add(layers.Bidirectional(layers.SimpleRNN(100, activation= 'relu', return_sequences=True, dropout = 0.5)))
model.add(layers.Bidirectional(layers.SimpleRNN(100, activation= 'relu', dropout = 0.5)))
model.add(layers.Dense(2, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 128
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 100)          2000000   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 250, 200)          40200     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 250, 200)          60200     
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 250, 200)          60200     
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 200)               60200     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 2,221,202
Trainable params: 2,221,202
Non-trainable params: 0
____________________________________________

In [17]:
# history = model.fit(pad_train, y_train, epochs=epochs, batch_size=batch_size)
history = model.fit(pad_train, y_train, epochs=epochs,validation_split=0.2)

Epoch 1/10
207/207 [==============================] - 611s 3s/step - loss: 4424.5435 - accuracy: 0.9088 - val_loss: 0.0180 - val_accuracy: 0.9958
Epoch 2/10
207/207 [==============================] - 585s 3s/step - loss: 0.0164 - accuracy: 0.9961 - val_loss: 0.0200 - val_accuracy: 0.9958
Epoch 3/10
207/207 [==============================] - 583s 3s/step - loss: 0.0068 - accuracy: 0.9988 - val_loss: 0.0207 - val_accuracy: 0.9964
Epoch 4/10
207/207 [==============================] - 584s 3s/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 0.0422 - val_accuracy: 0.9945
Epoch 5/10
207/207 [==============================] - 578s 3s/step - loss: 0.0151 - accuracy: 0.9982 - val_loss: 0.0490 - val_accuracy: 0.9952
Epoch 6/10
207/207 [==============================] - 577s 3s/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.0330 - val_accuracy: 0.9915
Epoch 7/10
207/207 [==============================] - 580s 3s/step - loss: 0.1091 - accuracy: 0.9964 - val_loss: 0.0233 - val_accuracy: 0.9